# Học **trọng số lớp** cho so khớp **Span Adaptation Vectors** (Train · GramVar/ParaVE)

- **Mục đích**
  - Từ các **Span Adaptation Vectors** (mỗi argument **ARG-k** có 12 vector lớp: `layer_1..layer_12`), học ra **trọng số theo lớp** để tối ưu phân biệt **cặp cùng nhãn** (positive: cùng ARG-k) và **khác nhãn** (negative: khác ARG-k).
  - Kết quả là 1 vector trọng số (chuẩn hoá softmax) dùng để **tổng hợp điểm tương đồng có trọng số theo lớp** khi tìm kiếm / matching về sau.

- **Đầu vào**
  - Thư mục Span AV **Train**:
    ```
    .../span_adaptation_vectors_train_gramvar/<verb_name>/sentence_<i>_ARG_<k>.pt
    .../span_adaptation_vectors_train_parave/<verb_name>/sentence_<i>_ARG_<k>.pt
    ```
    > Mỗi file `.pt` là **dict 12 lớp**: `{"layer_1": Tensor[2304], ..., "layer_12": Tensor[2304]}` (2304 = concat begin/end/content).



- **Đầu ra**
  - Thư mục lưu kết quả:
    ```
    .../matching_results/learned_layer_weights.pt
    ```
  - Một vector **trọng số 12 chiều** (đã **softmax**) tương ứng `layer_1..layer_12`.


- **Quy trình**
  1. **Nạp toàn bộ** span vectors Train từ hai bộ GramVar + ParaVE.
  2. **Tạo tập huấn luyện cặp** (mặc định `NUM_PAIRS_TO_SAMPLE = 50_000`):
     - 50% **positive**: hai span chung `arg_label` (vd. cùng `ARG-0`).
     - 50% **negative**: hai span khác `arg_label`.
     - Với mỗi cặp, tính **cosine similarity** cho **12 lớp** → vector đặc trưng 12 chiều.
  3. **Huấn luyện** mô hình tuyến tính `Linear(12→1)` với **BCEWithLogitsLoss** để dự đoán xác suất *cùng nhãn*.
  4. **Rút trọng số** từ lớp linear → **chuẩn hoá Softmax** để có phân phối trọng số theo lớp.
  5. **Lưu** trọng số ra `learned_layer_weights.pt`.


In [ ]:
# Import các thư viện cần thiết
import torch
import os
import json
import glob
from tqdm.auto import tqdm
import torch.nn.functional as F
import random

# --- 1. CẤU HÌNH ---
NUM_PAIRS_TO_SAMPLE = 50000
LEARNING_RATE = 0.01
NUM_EPOCHS = 100
NUM_LAYERS_TO_WEIGHT = 12

# --- THIẾT LẬP CÁC ĐƯỜNG DẪN ---
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dữ liệu Train (Span Adaptation Vectors)
train_datasets_dirs = [
    os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Train_Content/span_adaptation_vectors_train_gramvar'),
    os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Train_Content/span_adaptation_vectors_train_parave'),
]

# Nơi lưu kết quả
output_dir = os.path.join(drive_base_path, 'matching_results')
os.makedirs(output_dir, exist_ok=True)
output_weights_file = os.path.join(output_dir, 'learned_layer_weights.pt')


# --- 2. CÁC HÀM XỬ LÝ  ---

def load_all_training_data(span_av_dirs):
    """
    Tải toàn bộ span vector bằng cách quét các thư mục.
    """
    print("--- Bước 1: Tải toàn bộ dữ liệu train vào bộ nhớ... ---")
    all_spans = []

    for span_dir in span_av_dirs:
        # Quét tất cả các file .pt trong thư mục và các thư mục con
        all_files = glob.glob(os.path.join(span_dir, '**', '*.pt'), recursive=True)

        for file_path in tqdm(all_files, desc=f"Đọc {os.path.basename(span_dir)}"):
            try:
                # Tên file có dạng: sentence_i_ARG_X.pt
                filename_parts = os.path.basename(file_path).replace('.pt', '').split('_')
                arg_label = f"{filename_parts[-2]}-{filename_parts[-1]}" # Tái tạo lại ARG-X

                # Tải dữ liệu vector (đây là dict của các lớp cho 1 argument)
                layers_data = torch.load(file_path, map_location='cpu')

                all_spans.append({
                    'arg_label': arg_label,
                    'vectors': layers_data
                    # Các thông tin khác như verb_name, sentence_text không cần thiết cho bước này
                })
            except Exception as e:
                # Bỏ qua nếu có lỗi
                pass
    return all_spans

# Các hàm create_similarity_dataset, LayerWeightModel, train_model không thay đổi
def create_similarity_dataset(all_spans, num_pairs, num_layers):
    print(f"\n--- Bước 2: Tạo bộ dữ liệu với {num_pairs} cặp để học trọng số... ---")
    features, labels = [], []
    spans_by_label = {}
    for span in all_spans:
        label = span['arg_label']
        spans_by_label.setdefault(label, []).append(span)

    all_labels = list(spans_by_label.keys())

    for _ in tqdm(range(num_pairs), desc="Tạo cặp dữ liệu"):
        is_positive_pair = random.random() < 0.5
        if is_positive_pair:
            label = random.choice([l for l, s in spans_by_label.items() if len(s) > 1])
            span1, span2 = random.sample(spans_by_label[label], 2)
            labels.append(1.0)
        else:
            label1, label2 = random.sample(all_labels, 2)
            span1, span2 = random.choice(spans_by_label[label1]), random.choice(spans_by_label[label2])
            labels.append(0.0)

        sim_scores = [F.cosine_similarity(span1['vectors'][f'layer_{i+1}'].unsqueeze(0), span2['vectors'][f'layer_{i+1}'].unsqueeze(0)) for i in range(num_layers)]
        features.append(torch.cat(sim_scores))

    return torch.stack(features), torch.tensor(labels).unsqueeze(1)

class LayerWeightModel(torch.nn.Module):
    def __init__(self, num_layers):
        super().__init__()
        self.linear = torch.nn.Linear(num_layers, 1)
    def forward(self, x):
        return self.linear(x)

def train_model(model, features, labels, lr, epochs):
    print("\n--- Bước 3: Bắt đầu huấn luyện mô hình... ---")
    model.to(device)
    features, labels = features.to(device), labels.to(device)
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    pbar = tqdm(range(epochs), desc="Huấn luyện")
    for epoch in pbar:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 10 == 0:
            pbar.set_postfix_str(f"Loss: {loss.item():.4f}")
    print(" -> Huấn luyện hoàn tất!")

# --- 3. THỰC THI CHƯƠNG TRÌNH ---
if __name__ == "__main__":
    all_training_spans = load_all_training_data(train_datasets_dirs)
    X_train, y_train = create_similarity_dataset(all_training_spans, NUM_PAIRS_TO_SAMPLE, NUM_LAYERS_TO_WEIGHT)
    layer_model = LayerWeightModel(NUM_LAYERS_TO_WEIGHT)
    train_model(layer_model, X_train, y_train, LEARNING_RATE, NUM_EPOCHS)

    print("\n--- Bước 4: Trích xuất và hiển thị trọng số... ---")
    raw_weights = layer_model.linear.weight.data.squeeze(0).cpu()
    normalized_weights = F.softmax(raw_weights, dim=0)

    print("\nTrọng số quan trọng (đã được chuẩn hóa bằng Softmax):")
    for i in range(NUM_LAYERS_TO_WEIGHT):
        print(f"  - layer_{i+1}: {normalized_weights[i].item():.4f}")

    torch.save(normalized_weights, output_weights_file)
    print(f"\nĐã lưu các trọng số đã học vào: {output_weights_file}")


--- Bước 1: Tải toàn bộ dữ liệu train vào bộ nhớ... ---


Đọc span_adaptation_vectors_train_gramvar:   0%|          | 0/10390 [00:00<?, ?it/s]

Đọc span_adaptation_vectors_train_parave:   0%|          | 0/5131 [00:00<?, ?it/s]


--- Bước 2: Tạo bộ dữ liệu với 50000 cặp để học trọng số... ---


Tạo cặp dữ liệu:   0%|          | 0/50000 [00:00<?, ?it/s]


--- Bước 3: Bắt đầu huấn luyện mô hình... ---


Huấn luyện:   0%|          | 0/100 [00:00<?, ?it/s]

 -> Huấn luyện hoàn tất!

--- Bước 4: Trích xuất và hiển thị trọng số... ---

Trọng số quan trọng (đã được chuẩn hóa bằng Softmax):
  - layer_1: 0.0543
  - layer_2: 0.0615
  - layer_3: 0.0409
  - layer_4: 0.0562
  - layer_5: 0.0792
  - layer_6: 0.0718
  - layer_7: 0.1003
  - layer_8: 0.0833
  - layer_9: 0.1139
  - layer_10: 0.0942
  - layer_11: 0.0994
  - layer_12: 0.1449

Đã lưu các trọng số đã học vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/matching_results/learned_layer_weights.pt


- **Tham số chính**
  - `NUM_PAIRS_TO_SAMPLE = 50_000` – số cặp sinh dữ liệu học.
  - `NUM_LAYERS_TO_WEIGHT = 12` – số lớp dùng (layer_1..layer_12).
  - `LEARNING_RATE = 0.01`, `NUM_EPOCHS = 100`.
  - Tự động dùng **GPU** nếu có (`cuda`).

- **Cách dùng trọng số sau khi học**
  - Cho hai span A, B: tính `cos_sim_l = cosine(A[layer_l], B[layer_l])` với `l = 1..12`.
  - Điểm cuối:  
$$
\text{score} = \sum_{l=1}^{12} w_l \cdot \mathrm{cos\_sim}_{\,l}
$$

    trong đó $w_l$ là trọng số đã học (softmax).
